In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import csv


In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
There is no [win32] chromedriver for browser 90.0.4430 in cache
Get LATEST driver version for 90.0.4430


In [ ]:
# URL of page to be scraped nasa mars news
url = 'https://finances.worldbank.org/Other/IEG-World-Bank-Project-Performance-Ratings/rq9d-pctf/data'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify())

In [ ]:
# get news title, news link, news text, news date
results = soup.find_all("div", attrs={"class":"socrata-table"})

In [ ]:
table = results[0].find("table")

In [ ]:
body = table.find_all("tr")
head=body[0]
body_rows=body[1:]
headings=[]
for item in head.find_all("th"):
    item=(item.text).rstrip("\n")
    headings.append(item)
#print(headings)

In [ ]:
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [ ]:
project_df = pd.DataFrame(data=all_rows,columns=headings)

In [ ]:
#print(df.columns)

In [ ]:
project_df.columns=['Project ID','Project Name','Region','Country Code','Country Name','Approval Date',
       'Approval FY','Sector Board','Agreement Type','Lending Project Cost','Net Commitment',
       'Deactivation Date','Exit FY','Lending Instrument Typen','Lending Instrument','Product Line Code',
       'Product Line','IEG_EvalDate','IEG_EvalFY','IEG_EvalType','ERR at Appraisal','ERR at Completion',
       'IEG_Outcome','IEG_RDO','(disc)IEG_IDImpact','IEG_BankQualityAtEntry','IEG_BankQualityOfSupervision',
       'IEG_OverallBankPerf','(disc)IEG_BorrPrep','EG_ImplementingAgencyPerf','IEG_GovernmentPerf',
       'IEG_OverallBorrPerf','IEG_ICRQuality','(disc)IEG_Sustainability','IEG_MEQualit','IEG_SourceDocumentURL']
#project_df

In [ ]:
project_df=project_df.loc[:,['Project ID','Project Name','Region','Country Code','Country Name','Approval Date',
       'Approval FY','Sector Board','Lending Project Cost','Net Commitment',
       'Deactivation Date','Lending Instrument','Product Line Code',
       'Product Line','IEG_Outcome','IEG_SourceDocumentURL']]
#project_df

In [ ]:
project_df.to_html('app/templates/iegData.html')

In [ ]:
browser.quit()

In [ ]:
gdp_df=pd.read_csv('data/gdp-per-capita-worldbank.csv')
#gdp_df.head(20)

In [ ]:
gdp_df.to_html('app/templates/gdpData.html')

In [ ]:
population_df=pd.read_csv('data/world-population-by-world-regions-post-1820.csv')
#population_df.head(20)

In [ ]:
population_df.to_html('app/templates/populationData.html')

In [ ]:
cpi_df=pd.read_csv('data/merged_cpi_data.csv')
#cpi_df.head(20)

In [ ]:
cpi_df.to_html('app/templates/cpiData.html')